In [51]:
#!/usr/bin/python
# -*- coding:utf-8 -*-
#standard python packages
import numpy as np
import matplotlib.pyplot as plt
import math
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.animation import FuncAnimation


def update(q):    #function for the animation
    label = 'timestep {0}'.format(q+1)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    temp = np.flipud(counts[q].reshape(48,48))
    im = ax.matshow(temp, cmap='gray')
    ax.set_title(label)
    ax.xaxis.set_ticks_position('bottom')    #by default in matshow() the x-axis is located at top and I prefer the bottom
    ax.invert_yaxis()    #the aforementioned invertion
    cax.cla()    #clear the color bar between each timestep
    plt.colorbar(im, cax=cax)
    return im, ax

filename = "elve_mag-5_nobg"    #this program is still only able to run single files
with open("text_files/" + filename + ".txt") as f:
    data = []
    for line in f:
        line = line.split()
        if line:
            line = [int(i) for i in line]
            data.append(line)    #making the text file into a list

length = np.shape(data)[0]    #number of rows in the text file
ind = range(0, length, 49)    #list of numbers with the indices in the list data indicating change in timestep, [0, 49, 98, 147....]
nz = int(len(ind))    #nz is the total number of timesteps in the textfile
counts = np.zeros((nz, 2304))    #list of nz lists each with 2304 zeros corresponding to each pixel
for iz in range(nz):
    counts[iz][:] = np.concatenate((data[ind[iz] + 1 : ind[iz] + 49]), axis = 0)    #filling the nz lists with the data


In [52]:
##################################################
#changeable parameters

timestamp = 4    #number between 0 and nz, have to run and see what nz is or check text file -----> label will be incremented once
#timestamp is only relevant if stampnr is not 0 or 1
stampnr = 0    #0 for plotting sum of all timestamps, 1 for animated all the timestamps any thing else will plotting timestamp number "timestamp"
#In the case of wanting to use unintegrated text files there will be too many frames for animation to be reasonable, even just saving it will take a long time
interv = 150    #interval for changing frames in the animation in milliseconds
savename = "plots/" + filename    #this will be the name the plot or animation is saved as, no file ending
dots = 100    #dots per inch for the animation
save_show = 0    #0 = only save plot, 1 = both save and show plot, any other number will just show the plot

##################################################


fig = plt.figure()
ax = plt.gca()
divider = make_axes_locatable(ax)    #this is for the colorbar
cax = divider.append_axes("right", size="5%", pad=0.1)
temp = np.zeros((48, 48))    #making an array of zeros that is going to be filled with data
if stampnr == 0:
    label = 'Plot of the sums of all timestamps'
    for a in counts:    #this will put all the timestamps in one plot
        temp = np.add(temp, np.flipud(a.reshape(48,48)))
    im = ax.matshow(temp, cmap='gray')
    ax.set_title(label)
    ax.xaxis.set_ticks_position('bottom')    #by default in matshow() the x-axis is located at top and I prefer the bottom
    ax.invert_yaxis()    #the aforementioned invertion
    plt.colorbar(im, cax=cax)
    if save_show == 0 or save_show == 1:
        fig.savefig(savename + ".eps")
elif stampnr == 1:
    anim = FuncAnimation(fig, update, frames=np.arange(0, nz), interval=interv)
    if save_show == 0 or save_show == 1:
        anim.save(savename + ".gif", dpi=dots, writer="imagemagick")
else:
    label = 'timestep {0}'.format(timestamp+1)
    rowchange = 0    #placeholder to see when to switch row
    row = 0    #placeholder for the current row
    temp = np.flipud(counts[timestamp].reshape(48,48))
    im = ax.matshow(temp, cmap='gray')
    ax.set_title(label)
    ax.xaxis.set_ticks_position('bottom')    #by default in matshow() the x-axis is located at top and I prefer the bottom
    ax.invert_yaxis()    #the aforementioned invertion
    plt.colorbar(im, cax=cax)
    if save_show == 0 or save_show == 1:
        fig.savefig(savename + "_" + str(timestamp) + "_.eps")

if save_show != 0:
    plt.show()